In [1]:
from sklearn.datasets import load_iris,load_breast_cancer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score ,recall_score, precision_score

C:\Users\ale_g\AppData\Roaming\Python\Python36\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
data.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'target'],
      dtype='object')

In [3]:
data = pd.DataFrame(load_breast_cancer().data,columns=load_breast_cancer().feature_names)
data['target'] = load_breast_cancer().target
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('target',axis=1), data.target, test_size=0.33, random_state=42)

In [47]:
class RandomModelsClassifier():
    
    def __init__(self, model, n_estimators = 10, row_sample_ratio = 0.666, col_sample_ratio = '',random_state = '',
                amostragem_rep = False, random_param = False, params = {}):
        self.n_estimators = n_estimators
        self.model = model
        self.row_sample_ratio = row_sample_ratio
        self.col_sample_ratio = col_sample_ratio
        self.random_state = random_state
        self.amostragem_rep = amostragem_rep
        self.random_param = random_param
        self.params = params
        self.forests = []
 
    def fit(self,X_train,y_train):
        from math import sqrt
        import numpy as np
        from copy import deepcopy
        
        if self.random_state != "":
            np.random.seed(self.random_state)
            
        self.forests = []
        self.col = []
        for n in range(0,self.n_estimators):
            if self.col_sample_ratio == '':
                samp_col = np.random.choice(X_train.columns,size = int(round(sqrt(len(X_train.columns)),0)),replace=self.amostragem_rep)
            else:
                samp_col = np.random.choice(X_train.columns,size = int(round(self.col_sample_ratio*(len(X_train.columns)),0)),replace=self.amostragem_rep)
                
            samp_lines = np.random.choice(X_train.index,size = int(round(self.row_sample_ratio*len(X_train),0)),replace=self.amostragem_rep)
            
            if self.random_param:
                new_dict = {}
                for i in self.params:
                    new_dict[i] = np.random.choice(self.params[i],1)[0]
                    self.model.set_params(**new_dict)
            
            model = self.model.fit(X_train.loc[samp_lines][samp_col], y_train.loc[samp_lines][:])
            
            self.forests.append(deepcopy(model))
            self.col.append(samp_col)
            
   
    def predict(self,X_test):
        from collections import Counter
        self.pred_final = []
        self.pred = []
        #para cada modelo da predict com as colunas que foram usadas anteriormente no modelo fittado
        for cont, modelos in enumerate(self.forests):
            self.pred.append(modelos.predict(X_test.loc[:][self.col[cont]]))
            
        self.pred = pd.DataFrame(self.pred)
        ind = None
        for c in range(0,len(self.pred.columns)):
            ind , _ = Counter(self.pred.iloc[:][c].values).most_common()[0]
            self.pred_final.append(ind)
        return(self.pred_final)
    
    def predict_proba(self,X_test):
        from collections import Counter
        self.pred_final = []
        for num_linhas in X_test.index:
            self.pred = []
            for cont,modelos in enumerate(self.forests):
                self.pred.append(modelos.predict(pd.DataFrame(X_test.loc[num_linhas][self.col[cont]]).T)[0])
            c = Counter(self.pred)
            dicio_prob = {}
            for indexes, valores in c.most_common():
                dicio_prob[indexes] = valores/self.n_estimators 
            self.pred_final.append(dicio_prob)
        unicos = self.forests[0].classes_
        self.pred_desejado = [[] for i in range(0,len(pred))]
        for cont,i in enumerate(self.pred_final):
            try:
                self.pred_desejado[cont].append(i[unicos[0]])
            except:
                self.pred_desejado[cont].append(0.00)
                pass
            try:
                self.pred_desejado[cont].append(i[unicos[1]])
            except:
                self.pred_desejado[cont].append(0.00)
    
        return(self.pred_desejado)


In [48]:
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

rm = RandomModelsClassifier(DecisionTreeClassifier(random_state=42),
                            10,row_sample_ratio=0.8,random_state=42,random_param = True,
                           params = {'max_depth':[1,3,5,7],'min_samples_split':[3,5,7,9], 'min_samples_leaf':[1,3,5,7]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.9680851063829787

In [51]:
probs = rm.predict_proba(X_test)
probs[:5]

[[0.1, 0.9], [0.9, 0.1], [0.9, 0.1], [0.0, 1.0], [0.0, 1.0]]

In [454]:
accuracy_score(y_test,RandomForestClassifier(random_state=42).fit(X_train,y_train).predict(X_test))

0.9414893617021277

In [430]:
from sklearn.model_selection import GridSearchCV

In [431]:
grid = GridSearchCV(RandomForestClassifier(random_state=42),
            {'max_depth':[1,3,5,7],'min_samples_split':[3,5,7,9], 'min_samples_leaf':[1,3,5,7]},
            make_scorer(accuracy_score))

In [455]:
grid.fit(X_train,y_train);

In [445]:
grid.best_params_

{'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 3}

In [446]:
rf = RandomForestClassifier(random_state=42, max_depth=5,min_samples_leaf=3,min_samples_split=3)

In [456]:
rf.fit(X_train,y_train)
accuracy_score(y_test,rf.predict(X_test))

0.9574468085106383

In [508]:
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

rm = RandomModelsClassifier(DecisionTreeClassifier(random_state=42),
                            10,row_sample_ratio=0.8,random_state=42,random_param = False,
                           params = {'max_depth':[1,3,5,7],'min_samples_split':[3,5,7,9], 'min_samples_leaf':[1,3,5,7]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.9414893617021277

<h1>KNN</h1>

In [517]:
grid_knn = GridSearchCV(KNeighborsClassifier(),
                        param_grid={'n_neighbors':[2,3,5,7,9]},
                        scoring=make_scorer(accuracy_score))
grid_knn.fit(X_train,y_train);
knn = KNeighborsClassifier().set_params(**grid_knn.best_params_)
knn.fit(X_train,y_train)
accuracy_score(y_test,knn.predict(X_test))

0.973404255319149

In [557]:
rm = RandomModelsClassifier(KNeighborsClassifier(n_neighbors=9),
                            10,row_sample_ratio=0.8,
                            random_state=42,random_param = False,
                           params = {'n_neighbors':[2,3,5,7,9]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.9574468085106383

In [558]:
rm = RandomModelsClassifier(KNeighborsClassifier(),
                            10,row_sample_ratio=0.8,
                            random_state=42,random_param = True,
                           params = {'n_neighbors':[2,3,5,7,9]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.9627659574468085

In [565]:
rm = RandomModelsClassifier(KNeighborsClassifier(),
                            10,row_sample_ratio=1,
                            random_state=42,random_param = True,
                           params = {'n_neighbors':[2,3,5,7,9]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.9627659574468085

<h1>SVM</h1>

In [568]:
grid_svc = GridSearchCV(SVC(random_state=42),
                        param_grid={'C':[0.1,0.5,0.7,0.8,0.9,1]},
                        scoring=make_scorer(accuracy_score))
grid_svc.fit(X_train,y_train);
svc = SVC().set_params(**grid_svc.best_params_)
svc.fit(X_train,y_train)
accuracy_score(y_test,svc.predict(X_test))

0.6436170212765957

In [576]:
rm = RandomModelsClassifier(SVC(C=0.1),
                            10,row_sample_ratio=0.8,
                            random_state=42,random_param = False,
                           params={'C':[0.1,0.5,0.7,0.8,0.9,1]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.6436170212765957

In [566]:
rm = RandomModelsClassifier(SVC(),
                            10,row_sample_ratio=0.8,
                            random_state=42,random_param = True,
                           params={'C':[0.1,0.5,0.7,0.8,0.9,1]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.7180851063829787

In [567]:
rm = RandomModelsClassifier(SVC(),
                            10,row_sample_ratio=1,
                            random_state=42,random_param = True,
                           params={'C':[0.1,0.5,0.7,0.8,0.9,1]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.7393617021276596

<h1>Logistic Regression</h1>

In [582]:
from sklearn.linear_model import LogisticRegression
grid_log = GridSearchCV(LogisticRegression(random_state=42),
                        param_grid={'C':[0.1,0.5,0.7,0.8,0.9,1], 'penalty':['l2','l1']},
                        scoring=make_scorer(accuracy_score))
grid_log.fit(X_train,y_train);
log = LogisticRegression().set_params(**grid_log.best_params_)
log.fit(X_train,y_train)
accuracy_score(y_test,log.predict(X_test))

0.9627659574468085

In [589]:
rm = RandomModelsClassifier(LogisticRegression(C=0.7, penalty='l1'),
                            1000,row_sample_ratio=0.8,
                            random_state=42,random_param = False,
                           params={'C':[0.1,0.5,0.7,0.8,0.9,1]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.9680851063829787

In [594]:
rm = RandomModelsClassifier(LogisticRegression(),
                            1000,row_sample_ratio=0.8,
                            random_state=42,random_param = True,
                           params={'C':[0.1,0.5,0.7,0.8,0.9,1], 'penalty':['l2','l1']})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.9521276595744681

In [591]:
rm = RandomModelsClassifier(LogisticRegression(),
                            100,row_sample_ratio=1,
                            random_state=42,random_param = True,
                           params={'C':[0.1,0.5,0.7,0.8,0.9,1]})

rm.fit(X_train,y_train)
accuracy_score(y_test,rm.predict(X_test))

0.9627659574468085

In [579]:
rm.forests

[LogisticRegression(C=0.8, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state